In [1]:
import sys
import os

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import sys
sys.path.append("..")

from IPython.display import display, Markdown
import numpy as np
import pandas as pd

from forcateri import TimeSeries

def mprint(s): display(Markdown(s))

In [3]:
import logging

# Get the logger you used in your TimeSeries class
logger = logging.getLogger("forcateri.data.timeseries")
logger.setLevel(logging.INFO)  # or logging.DEBUG for more verbosity

# Create handler if it doesn't exist
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# Optional: prevent propagation to avoid double logs
logger.propagate = False


In [4]:
%reload_ext autoreload

In [5]:
df = pd.read_csv("/home/user/DFKI/forcateri/_data/hourly_data.csv",index_col=0)

In [6]:

df_978 = df[df['room_id'] == 978]
df_978 = df_978.drop(columns=['room_id'])
df_978['rounded_ts'] =pd.to_datetime(df_978['rounded_ts'])

In [7]:
df_978.set_index('rounded_ts', inplace=True)

In [8]:
df_978

,sin_time_of_day,cos_time_of_day,sin_time_of_year,cos_time_of_year,day_of_week,max_temperature_1,max_temperature_2,outside_temp,room_temperature,delta
rounded_ts,,,,,,,,,,
2019-06-01 00:00:00,0.500000,1.000000,0.753708,0.069149,5,22.616923,22.709231,10.8,21.081304,0.0
2019-06-01 01:00:00,0.629410,0.982963,0.753708,0.069149,5,22.522000,22.626667,10.8,21.079091,0.0
2019-06-01 02:00:00,0.750000,0.933013,0.753708,0.069149,5,22.413846,22.545385,9.8,21.075185,0.0
2019-06-01 03:00:00,0.853553,0.853553,0.753708,0.069149,5,22.425000,22.540000,9.8,21.078500,0.0
2019-06-01 04:00:00,0.933013,0.750000,0.753708,0.069149,5,22.549231,22.601538,9.8,21.101111,0.0
...,...,...,...,...,...,...,...,...,...,...
2022-04-18 20:00:00,0.066987,0.750000,0.980075,0.360257,0,23.687273,25.089091,9.2,20.499355,0.0
2022-04-18 21:00:00,0.146447,0.853553,0.980075,0.360257,0,24.031538,25.581538,9.3,20.510811,0.0
2022-04-18 22:00:00,0.250000,0.933013,0.980075,0.360257,0,24.100556,25.333889,8.3,20.440870,0.0


In [9]:
ts = TimeSeries(data = df_978,representation='determ')

2025-05-28 17:24:30,563 - INFO - TimeSeries initialized from compatible-format DataFrame.


In [10]:
expected_index_names = ['offset', 'time_stamp']
expected_column_names = ['feature', 'representation']
TimeSeries.is_matching_format(df_978,representation='determ')

False

In [11]:
test = ts.get_feature_slice(index=['max_temperature_1','max_temperature_2','outside_temp']).data[:5000]
len(test)

2025-05-28 17:24:30,586 - INFO - TimeSeries initialized from internal-format DataFrame.


5000

In [12]:
rng = np.random.default_rng()
n_cols, n_rows = 3, 12
index = pd.date_range(start="2000-01-01", freq="h", periods=n_rows)


In [13]:
raw_df = pd.DataFrame(
    data=rng.random(n_cols * n_rows).reshape(n_rows, n_cols)
)
mprint("### Not compatible\nNo time information is provided:")
raw_df

### Not compatible
No time information is provided:

,0,1,2
0,0.004577,0.846473,0.670492
1,0.592441,0.299542,0.745016
2,0.013113,0.821635,0.998901
3,0.280232,0.113999,0.837425
4,0.890282,0.183013,0.069274
5,0.571761,0.677796,0.892244
6,0.469598,0.174764,0.797578
7,0.950704,0.660146,0.816759
8,0.995158,0.548194,0.707671
9,0.525651,0.446582,0.290692


In [14]:
dt_indexed_df = raw_df.copy()
dt_indexed_df.set_index(index, inplace=True)
mprint("### Compatible\nThe column index represents deterministic features, the row index represents time steps")
dt_indexed_df

### Compatible
The column index represents deterministic features, the row index represents time steps

,0,1,2
2000-01-01 00:00:00,0.004577,0.846473,0.670492
2000-01-01 01:00:00,0.592441,0.299542,0.745016
2000-01-01 02:00:00,0.013113,0.821635,0.998901
2000-01-01 03:00:00,0.280232,0.113999,0.837425
2000-01-01 04:00:00,0.890282,0.183013,0.069274
2000-01-01 05:00:00,0.571761,0.677796,0.892244
2000-01-01 06:00:00,0.469598,0.174764,0.797578
2000-01-01 07:00:00,0.950704,0.660146,0.816759
2000-01-01 08:00:00,0.995158,0.548194,0.707671
2000-01-01 09:00:00,0.525651,0.446582,0.290692


In [15]:
ambiguous_col_df = dt_indexed_df.copy()
ambiguous_col_df.columns = pd.MultiIndex.from_product([["delta"], [1, 5, 9]])
mprint("""
### Compatible but...\n
Unclear how to interpret the inner column index: As samples? As quantiles? Which quantiles?
Thus, the compatibility check should succeed but an error can still be thrown by the constructor
if `representation` and/or `quantiles` are not provided.
""")
ambiguous_col_df


### Compatible but...

Unclear how to interpret the inner column index: As samples? As quantiles? Which quantiles?
Thus, the compatibility check should succeed but an error can still be thrown by the constructor
if `representation` and/or `quantiles` are not provided.


delta                    
                            1         5         9
2000-01-01 00:00:00  0.004577  0.846473  0.670492
2000-01-01 01:00:00  0.592441  0.299542  0.745016
2000-01-01 02:00:00  0.013113  0.821635  0.998901
2000-01-01 03:00:00  0.280232  0.113999  0.837425
2000-01-01 04:00:00  0.890282  0.183013  0.069274
2000-01-01 05:00:00  0.571761  0.677796  0.892244
2000-01-01 06:00:00  0.469598  0.174764  0.797578
2000-01-01 07:00:00  0.950704  0.660146  0.816759
2000-01-01 08:00:00  0.995158  0.548194  0.707671
2000-01-01 09:00:00  0.525651  0.446582  0.290692
2000-01-01 10:00:00  0.077331  0.150334  0.170794
2000-01-01 11:00:00  0.667849  0.320119  0.123407

In [16]:
expected_column_names = ['feature', 'representation']
ambiguous_col_df.columns.names = expected_column_names
ambiguous_col_df

feature                 delta                    
representation              1         5         9
2000-01-01 00:00:00  0.004577  0.846473  0.670492
2000-01-01 01:00:00  0.592441  0.299542  0.745016
2000-01-01 02:00:00  0.013113  0.821635  0.998901
2000-01-01 03:00:00  0.280232  0.113999  0.837425
2000-01-01 04:00:00  0.890282  0.183013  0.069274
2000-01-01 05:00:00  0.571761  0.677796  0.892244
2000-01-01 06:00:00  0.469598  0.174764  0.797578
2000-01-01 07:00:00  0.950704  0.660146  0.816759
2000-01-01 08:00:00  0.995158  0.548194  0.707671
2000-01-01 09:00:00  0.525651  0.446582  0.290692
2000-01-01 10:00:00  0.077331  0.150334  0.170794
2000-01-01 11:00:00  0.667849  0.320119  0.123407

In [17]:
ts_ambiguous = TimeSeries(ambiguous_col_df,representation='quantile', quantiles = [0.1, 0.5, 0.9])

2025-05-28 17:24:30,673 - INFO - Reordering column names to match expected format.
2025-05-28 17:24:30,676 - INFO - TimeSeries initialized from compatible-format DataFrame.


In [18]:
ts_ambiguous.data

feature                        delta                    
representation                   0.1       0.5       0.9
offset time_stamp                                       
0 days 2000-01-01 00:00:00  0.004577  0.846473  0.670492
       2000-01-01 01:00:00  0.592441  0.299542  0.745016
       2000-01-01 02:00:00  0.013113  0.821635  0.998901
       2000-01-01 03:00:00  0.280232  0.113999  0.837425
       2000-01-01 04:00:00  0.890282  0.183013  0.069274
       2000-01-01 05:00:00  0.571761  0.677796  0.892244
       2000-01-01 06:00:00  0.469598  0.174764  0.797578
       2000-01-01 07:00:00  0.950704  0.660146  0.816759
       2000-01-01 08:00:00  0.995158  0.548194  0.707671
       2000-01-01 09:00:00  0.525651  0.446582  0.290692
       2000-01-01 10:00:00  0.077331  0.150334  0.170794
       2000-01-01 11:00:00  0.667849  0.320119  0.123407

In [19]:
dt_quant = dt_indexed_df.copy()
dt_quant

,0,1,2
2000-01-01 00:00:00,0.004577,0.846473,0.670492
2000-01-01 01:00:00,0.592441,0.299542,0.745016
2000-01-01 02:00:00,0.013113,0.821635,0.998901
2000-01-01 03:00:00,0.280232,0.113999,0.837425
2000-01-01 04:00:00,0.890282,0.183013,0.069274
2000-01-01 05:00:00,0.571761,0.677796,0.892244
2000-01-01 06:00:00,0.469598,0.174764,0.797578
2000-01-01 07:00:00,0.950704,0.660146,0.816759
2000-01-01 08:00:00,0.995158,0.548194,0.707671
2000-01-01 09:00:00,0.525651,0.446582,0.290692


In [20]:
QUANTILES = [0.1, 0.5, 0.9]
ts_q = TimeSeries(dt_quant,representation='quantiles', quantiles=QUANTILES)
ts_q.data

2025-05-28 17:24:30,713 - INFO - TimeSeries initialized from compatible-format DataFrame.


,0,1,2
2000-01-01 00:00:00,0.004577,0.846473,0.670492
2000-01-01 01:00:00,0.592441,0.299542,0.745016
2000-01-01 02:00:00,0.013113,0.821635,0.998901
2000-01-01 03:00:00,0.280232,0.113999,0.837425
2000-01-01 04:00:00,0.890282,0.183013,0.069274
2000-01-01 05:00:00,0.571761,0.677796,0.892244
2000-01-01 06:00:00,0.469598,0.174764,0.797578
2000-01-01 07:00:00,0.950704,0.660146,0.816759
2000-01-01 08:00:00,0.995158,0.548194,0.707671
2000-01-01 09:00:00,0.525651,0.446582,0.290692


In [21]:
dt_quant.columns = pd.MultiIndex.from_product([['target'],QUANTILES], names=expected_column_names)

In [22]:
import numpy as np
import pandas as pd

from forcateri import TimeSeries

rows = pd.MultiIndex.from_product(
    [[pd.Timedelta(0), pd.Timedelta(1, "h")], pd.date_range(start=pd.Timestamp("2000-01-01"), freq="h", periods=4)],
    names=["offset", "time_stamp"]
)
cols = ["feat0", "feat1"]

data = np.zeros(shape=(len(rows), len(cols)))

df = pd.DataFrame(index=rows, columns=cols, data=data)
print(df)

ts = TimeSeries(data=df)
print(ts.data)

print(ts.representation)

2025-05-28 17:24:30,749 - INFO - Check index structure
2025-05-28 17:24:30,751 - INFO - Index is MultiIndex with datetime values.
2025-05-28 17:24:30,752 - INFO - Reordering index names to match expected format.
2025-05-28 17:24:30,753 - INFO - Casting the index to datetime format
2025-05-28 17:24:30,757 - INFO - TimeSeries initialized from compatible-format DataFrame.


                                     feat0  feat1
offset          time_stamp                       
0 days 00:00:00 2000-01-01 00:00:00    0.0    0.0
                2000-01-01 01:00:00    0.0    0.0
                2000-01-01 02:00:00    0.0    0.0
                2000-01-01 03:00:00    0.0    0.0
0 days 01:00:00 2000-01-01 00:00:00    0.0    0.0
                2000-01-01 01:00:00    0.0    0.0
                2000-01-01 02:00:00    0.0    0.0
                2000-01-01 03:00:00    0.0    0.0
feature                             feat0 feat1
representation                      value value
offset          time_stamp                     
0 days 00:00:00 2000-01-01 00:00:00   0.0   0.0
                2000-01-01 01:00:00   0.0   0.0
                2000-01-01 02:00:00   0.0   0.0
                2000-01-01 03:00:00   0.0   0.0
0 days 01:00:00 2000-01-01 00:00:00   0.0   0.0
                2000-01-01 01:00:00   0.0   0.0
                2000-01-01 02:00:00   0.0   0.0
                2000

In [23]:
df

feat0  feat1
offset          time_stamp                       
0 days 00:00:00 2000-01-01 00:00:00    0.0    0.0
                2000-01-01 01:00:00    0.0    0.0
                2000-01-01 02:00:00    0.0    0.0
                2000-01-01 03:00:00    0.0    0.0
0 days 01:00:00 2000-01-01 00:00:00    0.0    0.0
                2000-01-01 01:00:00    0.0    0.0
                2000-01-01 02:00:00    0.0    0.0
                2000-01-01 03:00:00    0.0    0.0

In [24]:
TimeSeries.is_compatible_format(df,representation='determ')

2025-05-28 17:24:30,788 - INFO - Check index structure
2025-05-28 17:24:30,793 - INFO - Index is MultiIndex with datetime values.


True

In [25]:
asd = TimeSeries(df,representation='determ')
asd.data

2025-05-28 17:24:30,801 - INFO - Check index structure
2025-05-28 17:24:30,803 - INFO - Index is MultiIndex with datetime values.
2025-05-28 17:24:30,803 - INFO - Reordering index names to match expected format.
2025-05-28 17:24:30,804 - INFO - Casting the index to datetime format
2025-05-28 17:24:30,806 - INFO - TimeSeries initialized from compatible-format DataFrame.


feature                             feat0 feat1
representation                      value value
offset          time_stamp                     
0 days 00:00:00 2000-01-01 00:00:00   0.0   0.0
                2000-01-01 01:00:00   0.0   0.0
                2000-01-01 02:00:00   0.0   0.0
                2000-01-01 03:00:00   0.0   0.0
0 days 01:00:00 2000-01-01 00:00:00   0.0   0.0
                2000-01-01 01:00:00   0.0   0.0
                2000-01-01 02:00:00   0.0   0.0
                2000-01-01 03:00:00   0.0   0.0

In [26]:
df.index = df.index.set_levels(df.index.levels[1].astype(int),level=1)
df.columns = pd.MultiIndex.from_product([df.columns, ["value"]], names=["feature", "representation"])

In [27]:
df

feature                            feat0 feat1
representation                     value value
offset          time_stamp                    
0 days 00:00:00 946684800000000000   0.0   0.0
                946688400000000000   0.0   0.0
                946692000000000000   0.0   0.0
                946695600000000000   0.0   0.0
0 days 01:00:00 946684800000000000   0.0   0.0
                946688400000000000   0.0   0.0
                946692000000000000   0.0   0.0
                946695600000000000   0.0   0.0

In [28]:
qq = TimeSeries(df,representation='determ')
qq.data

2025-05-28 17:24:30,844 - INFO - Check index structure
2025-05-28 17:24:30,845 - INFO - Check columns MultiIndex structure. One caveat is that df.index is not DateTimeIndex, casting to datetime is done in align_format().
2025-05-28 17:24:30,846 - INFO - Reordering column names to match expected format.
2025-05-28 17:24:30,846 - INFO - Reordering index names to match expected format.
2025-05-28 17:24:30,847 - INFO - Casting the index to datetime format
2025-05-28 17:24:30,849 - INFO - TimeSeries initialized from compatible-format DataFrame.


feature                             feat0 feat1
representation                      value value
offset          time_stamp                     
0 days 00:00:00 2000-01-01 00:00:00   0.0   0.0
                2000-01-01 01:00:00   0.0   0.0
                2000-01-01 02:00:00   0.0   0.0
                2000-01-01 03:00:00   0.0   0.0
0 days 01:00:00 2000-01-01 00:00:00   0.0   0.0
                2000-01-01 01:00:00   0.0   0.0
                2000-01-01 02:00:00   0.0   0.0
                2000-01-01 03:00:00   0.0   0.0

In [29]:
qq.data.index.get_level_values('time_stamp').astype('datetime64[ns]')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:00:00',
               '2000-01-01 02:00:00', '2000-01-01 03:00:00',
               '2000-01-01 00:00:00', '2000-01-01 01:00:00',
               '2000-01-01 02:00:00', '2000-01-01 03:00:00'],
              dtype='datetime64[ns]', name='time_stamp', freq=None)

In [30]:
qq.data.index = pd.MultiIndex.from_arrays([qq.data.index.get_level_values('offset'), pd.to_datetime(qq.data.index.get_level_values('time_stamp'))], names=qq.data.index.names)
qq.data

feature                             feat0 feat1
representation                      value value
offset          time_stamp                     
0 days 00:00:00 2000-01-01 00:00:00   0.0   0.0
                2000-01-01 01:00:00   0.0   0.0
                2000-01-01 02:00:00   0.0   0.0
                2000-01-01 03:00:00   0.0   0.0
0 days 01:00:00 2000-01-01 00:00:00   0.0   0.0
                2000-01-01 01:00:00   0.0   0.0
                2000-01-01 02:00:00   0.0   0.0
                2000-01-01 03:00:00   0.0   0.0

In [7]:
import numpy as np
import pandas as pd

from forcateri import TimeSeries

rows = pd.MultiIndex.from_product(
    [[pd.Timedelta(0), pd.Timedelta(1, "h")], pd.date_range(start=pd.Timestamp("2000-01-01"), freq="h", periods=4)],
    names=["offset", "time_stamp"]
)
cols = pd.MultiIndex.from_product(
    [["feat0", "feat1"], [.1, .9]], 
    names=["feature", "representation"]
)
data = np.zeros(shape=(len(rows), len(cols)))

df = pd.DataFrame(index=rows, columns=cols, data=data)
print(df)

ts = TimeSeries(data=df, representation=TimeSeries.DETERM_REP)
print(ts.data)

print(ts.representation)

2025-05-30 14:58:23,009 - ERROR - Features are having multiple value cols


feature                             feat0      feat1     
representation                        0.1  0.9   0.1  0.9
offset          time_stamp                               
0 days 00:00:00 2000-01-01 00:00:00   0.0  0.0   0.0  0.0
                2000-01-01 01:00:00   0.0  0.0   0.0  0.0
                2000-01-01 02:00:00   0.0  0.0   0.0  0.0
                2000-01-01 03:00:00   0.0  0.0   0.0  0.0
0 days 01:00:00 2000-01-01 00:00:00   0.0  0.0   0.0  0.0
                2000-01-01 01:00:00   0.0  0.0   0.0  0.0
                2000-01-01 02:00:00   0.0  0.0   0.0  0.0
                2000-01-01 03:00:00   0.0  0.0   0.0  0.0


InvalidRepresentationFormat: Feature level in the DataFrame is not unique. This is required for deterministic representation.

In [ ]:
TimeSeries.is_matching_format(df, representation='determ')

False

In [4]:
import numpy as np
import pandas as pd

from forcateri import TimeSeries

rows = pd.MultiIndex.from_product(
    [[pd.Timedelta(0), pd.Timedelta(1, "h")], pd.date_range(start=pd.Timestamp("2000-01-01"), freq="h", periods=4)],
    names=["offset", "time_stamp"]
)
cols = pd.MultiIndex.from_product(
    [["feat0", "feat1"], ["q1", "q9"]], 
    names=["feature", "representation"]
)
data = np.random.randn(len(rows), len(cols))

df = pd.DataFrame(index=rows, columns=cols, data=data)
df
# print(df)

ts = TimeSeries(data=df, representation=TimeSeries.QUANTILE_REP, quantiles=[.1, .9])
print(ts.data)

print(ts.quantiles)
print(ts.representation)

2025-05-30 15:26:21,039 - INFO - Check index structure
2025-05-30 15:26:21,042 - INFO - Index is MultiIndex with datetime values.
2025-05-30 15:26:21,044 - INFO - Reordering column names to match expected format.
2025-05-30 15:26:21,047 - INFO - Reordering index names to match expected format.
2025-05-30 15:26:21,049 - INFO - Casting the index to datetime format
2025-05-30 15:26:21,059 - INFO - TimeSeries initialized from compatible-format DataFrame.


feature                                 feat0               feat1          
representation                            0.1       0.9       0.1       0.9
offset          time_stamp                                                 
0 days 00:00:00 2000-01-01 00:00:00 -1.840133 -1.563882  0.157122  0.079106
                2000-01-01 01:00:00  1.220682 -0.069647 -0.418557 -0.369243
                2000-01-01 02:00:00 -0.533976  0.741094  0.885898  0.862337
                2000-01-01 03:00:00 -1.460151 -0.157472  0.790226 -0.292281
0 days 01:00:00 2000-01-01 00:00:00  0.081538  0.785280  0.446871  1.274175
                2000-01-01 01:00:00 -0.267996 -1.542810  1.333061  0.350815
                2000-01-01 02:00:00 -0.852365  0.839075  1.441263  1.590557
                2000-01-01 03:00:00 -1.170061 -0.943481  1.153371 -0.159490
[0.1, 0.9]
quantile


In [7]:
#TimeSeries.is_compatible_format(df=df, representation='quantile')
(ts * 2).data

2025-05-30 15:26:36,139 - INFO - TimeSeries initialized from internal-format DataFrame.


feature                                 feat0               feat1          
representation                            0.1       0.9       0.1       0.9
offset          time_stamp                                                 
0 days 00:00:00 2000-01-01 00:00:00 -3.680266 -3.127763  0.314244  0.158212
                2000-01-01 01:00:00  2.441364 -0.139295 -0.837114 -0.738486
                2000-01-01 02:00:00 -1.067952  1.482189  1.771797  1.724674
                2000-01-01 03:00:00 -2.920302 -0.314945  1.580453 -0.584562
0 days 01:00:00 2000-01-01 00:00:00  0.163076  1.570559  0.893741  2.548351
                2000-01-01 01:00:00 -0.535992 -3.085619  2.666122  0.701630
                2000-01-01 02:00:00 -1.704731  1.678150  2.882525  3.181114
                2000-01-01 03:00:00 -2.340121 -1.886963  2.306742 -0.318979

In [5]:
df2 = df.copy()
ts2 = TimeSeries(data=df2, representation=TimeSeries.QUANTILE_REP, quantiles=[.1, .9])

2025-05-30 15:24:29,455 - INFO - Check index structure
2025-05-30 15:24:29,459 - INFO - Index is MultiIndex with datetime values.
2025-05-30 15:24:29,464 - INFO - Reordering column names to match expected format.
2025-05-30 15:24:29,467 - INFO - Reordering index names to match expected format.
2025-05-30 15:24:29,470 - INFO - Casting the index to datetime format
2025-05-30 15:24:29,480 - INFO - TimeSeries initialized from compatible-format DataFrame.


In [6]:
(ts - ts2).data

2025-05-30 15:24:34,170 - INFO - TimeSeries initialized from internal-format DataFrame.


feature                             feat0      feat1     
representation                        0.1  0.9   0.1  0.9
offset          time_stamp                               
0 days 00:00:00 2000-01-01 00:00:00   0.0  0.0   0.0  0.0
                2000-01-01 01:00:00   0.0  0.0   0.0  0.0
                2000-01-01 02:00:00   0.0  0.0   0.0  0.0
                2000-01-01 03:00:00   0.0  0.0   0.0  0.0
0 days 01:00:00 2000-01-01 00:00:00   0.0  0.0   0.0  0.0
                2000-01-01 01:00:00   0.0  0.0   0.0  0.0
                2000-01-01 02:00:00   0.0  0.0   0.0  0.0
                2000-01-01 03:00:00   0.0  0.0   0.0  0.0

In [7]:
ts2.data * 2

feature                                 feat0               feat1          
representation                            0.1       0.9       0.1       0.9
offset          time_stamp                                                 
0 days 00:00:00 2000-01-01 00:00:00  0.201983 -0.612546 -3.793677  2.314899
                2000-01-01 01:00:00 -2.293448  1.900419  2.749458 -3.809626
                2000-01-01 02:00:00  0.421913 -0.556470 -0.414664 -0.705846
                2000-01-01 03:00:00 -0.630840  1.808349 -1.863930 -0.843786
0 days 01:00:00 2000-01-01 00:00:00  4.644125 -0.331005 -0.483987  1.530037
                2000-01-01 01:00:00 -3.268968 -2.284912 -1.824938  2.726032
                2000-01-01 02:00:00  2.123917 -3.572177  0.744019  0.022228
                2000-01-01 03:00:00 -3.724019  0.309588 -2.397858  0.262657

In [8]:
ts2.data

feature                                 feat0               feat1          
representation                            0.1       0.9       0.1       0.9
offset          time_stamp                                                 
0 days 00:00:00 2000-01-01 00:00:00  0.100992 -0.306273 -1.896839  1.157449
                2000-01-01 01:00:00 -1.146724  0.950210  1.374729 -1.904813
                2000-01-01 02:00:00  0.210957 -0.278235 -0.207332 -0.352923
                2000-01-01 03:00:00 -0.315420  0.904175 -0.931965 -0.421893
0 days 01:00:00 2000-01-01 00:00:00  2.322063 -0.165502 -0.241993  0.765018
                2000-01-01 01:00:00 -1.634484 -1.142456 -0.912469  1.363016
                2000-01-01 02:00:00  1.061958 -1.786089  0.372010  0.011114
                2000-01-01 03:00:00 -1.862010  0.154794 -1.198929  0.131329

In [3]:
import pickle 

with open("/home/user/DFKI/forcateri/_data/historicals_5ts.pkl", "rb") as f:
    historicals = pickle.load(f)

In [5]:
historicals[0][0]

<TimeSeries (DataArray) (time_stamp: 5, component: 3, sample: 1)> Size: 120B
array([[[-0.00232804],
        [-0.0048794 ],
        [ 0.02421307]],

       [[-0.00234671],
        [-0.00490786],
        [ 0.02484511]],

       [[-0.00220614],
        [-0.00484262],
        [ 0.00850878]],

       [[-0.00223677],
        [-0.00483329],
        [ 0.00867606]],

       [[-0.00230061],
        [-0.00483693],
        [ 0.0088593 ]]])
Coordinates:
  * time_stamp  (time_stamp) datetime64[ns] 40B 2020-01-14T21:00:00 ... 2020-...
  * component   (component) <U11 132B 'q_hca_q0.10' 'q_hca_q0.50' 'q_hca_q0.90'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
    metadata:           None

In [12]:
historicals[0][0].to_dataframe()

component,q_hca_q0.10,q_hca_q0.50,q_hca_q0.90
time_stamp,,,
2020-01-14 21:00:00,-0.002328,-0.004879,0.024213
2020-01-14 22:00:00,-0.002347,-0.004908,0.024845
2020-01-14 23:00:00,-0.002206,-0.004843,0.008509
2020-01-15 00:00:00,-0.002237,-0.004833,0.008676
2020-01-15 01:00:00,-0.002301,-0.004837,0.008859


In [16]:
dum = historicals[0][1].to_dataframe()

In [34]:
dd = historicals[0][0].to_dataframe()
dd_ts = TimeSeries(data=dd, representation=TimeSeries.QUANTILE_REP, quantiles=[0.1, 0.5, 0.9])

In [4]:
from forcateri.model.dartsmodels.dartsmodeladapter import DartsModelAdapter

In [18]:
from forcateri import TimeSeries

dum_ts = TimeSeries(data=dum, representation=TimeSeries.QUANTILE_REP, quantiles=[0.1, 0.5, 0.9])

In [35]:
new_offsets = [pd.Timedelta(i, "h") for i in range(1,len(dum_ts.data.index)+1)]
dum_ts.data.index = pd.MultiIndex.from_arrays(
    [new_offsets, dum_ts.data.index.get_level_values(1)],
    names=dum_ts.data.index.names
)

In [37]:
dum_ts.data

feature                                target                    
representation                            0.1       0.5       0.9
offset          time_stamp                                       
0 days 01:00:00 2020-01-14 22:00:00 -0.002345 -0.004907  0.024864
0 days 02:00:00 2020-01-14 23:00:00 -0.002371 -0.004923  0.024575
0 days 03:00:00 2020-01-15 00:00:00 -0.002237 -0.004833  0.008670
0 days 04:00:00 2020-01-15 01:00:00 -0.002270 -0.004829  0.008863
0 days 05:00:00 2020-01-15 02:00:00 -0.002302 -0.004813  0.009004

In [36]:
new_offsets_dd = [pd.Timedelta(i, "h") for i in range(1,len(dd_ts.data.index)+1)]
dd_ts.data.index = pd.MultiIndex.from_arrays(
    [new_offsets_dd, dd_ts.data.index.get_level_values(1)],
    names=dd_ts.data.index.names
)
dd_ts.data

feature                                target                    
representation                            0.1       0.5       0.9
offset          time_stamp                                       
0 days 01:00:00 2020-01-14 21:00:00 -0.002328 -0.004879  0.024213
0 days 02:00:00 2020-01-14 22:00:00 -0.002347 -0.004908  0.024845
0 days 03:00:00 2020-01-14 23:00:00 -0.002206 -0.004843  0.008509
0 days 04:00:00 2020-01-15 00:00:00 -0.002237 -0.004833  0.008676
0 days 05:00:00 2020-01-15 01:00:00 -0.002301 -0.004837  0.008859